In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import CRS
import cartopy.crs as ccrs
from tqdm import tqdm
from shapely import Point, LineString, Polygon
import numpy as np
from itertools import product

In [2]:
qm = 10_001_965.729
actual_poles = gpd.GeoDataFrame({'pole':['original','EPIA1','EPIA2'],
                                 'geometry':[Point(86+40/60, 46+17/60),
                                             Point(82.19,44.29),
                                             Point(88.14,45.28)]},
                                crs=4326)

In [3]:
# make circle
def make_circle_and_points(d, aeqd, num=20):
    circle = Point(0,0).buffer(d)
    minx,miny,maxx,maxy = circle.bounds
    x_points = np.linspace(minx,maxx,num)
    y_points = np.linspace(miny,maxy,num)

    points = gpd.GeoDataFrame(geometry=[Point(p) for p in product(x_points,y_points)], crs=aeqd).clip(circle)
    points['distance_to_centroid'] = points.distance(Point(0,0))
    points = points.sort_values(by='distance_to_centroid', ascending=True).reset_index(drop=True)

    # reproject and get lat lon coordinates of each point
    points_4326 = points.to_crs(4326)
    points_4326['lon'] = points_4326['geometry'].x
    points_4326['lat'] = points_4326['geometry'].y

    # then join with the points dataframe
    points = points.merge(points_4326[['lon','lat']],
                        left_index=True, right_index=True)

    points['proj4'] = points.apply(lambda row: f'+proj=aeqd +lon_0={row["lon"]} +lat_0={row["lat"]} +ellps=WGS84 +units=m +no_defs', axis=1)
    
    return points


def distance_to_boundary(proj4,poly_4326):
    '''
    # for each of these points, construct a new coordinates system centered on it...
    # reproject thepolygon to this coordinate system
    # get distance to boundary
    # there is probably a quicker way than this...but it takes ~24 seconds for 276 rows...and i think that is ok
    '''
    new_aeqd = CRS.from_proj4(proj4)
    rprj_poly = poly_4326.to_crs(new_aeqd)
    d = rprj_poly.geometry[0].exterior.distance(Point(0,0))
    
    return d/1000 # to put it in km

In [4]:
## read in data, filter on asia and largest contiguous landmass
world = gpd.read_file('../day3_polygons/ne_10m_admin_0_countries.zip')

# filter on asia (inc. Russia)
asia = world.loc[(world.REGION_UN == 'Asia') | (world.ADMIN=='Russia')]

# dissolve all polygons, then explode to single part to then identify 
# the biggest single polygon (i.e. the main Asian landmass (china, india, central asia...))
asia_dissolve = asia.dissolve()
asia_single_part = asia_dissolve.explode(index_parts=True).reset_index()

# use equal area projection to calculate & identify biggest polygon
hobo_dyer = CRS.from_proj4('+proj=cea +lon_0=0 +lat_ts=37.5 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs')
rprj_hd = asia_single_part.to_crs(hobo_dyer)
rprj_hd['area'] = rprj_hd.area
thepolygon = asia_single_part.loc[[rprj_hd['area'].idxmax()],'geometry'].values[0]  ## geometry of thepolygon is still in epsg4326
thepolygon_4326 = gpd.GeoSeries(thepolygon,crs=4326)

c:\Users\s1759665\Miniconda3\envs\shadow\Lib\site-packages\geopandas\geodataframe.py:1922: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[exploded_geom.name] = exploded_geom.values


In [5]:
## set up for first run

# largest area, is hereafter referred to as thepolygon
#crudely estimating centroid in geographic coordinates
# which will be wrong...because geographci coordinates
# and this will give a warning
x0 = thepolygon.centroid.x
y0 = thepolygon.centroid.y
centroid = Point(x0,y0)

# construct an azimuthal equidistant projection centered on the approximate 'centroid'
# the centroid will have coordinates, (0,0) in this projection
aeqd = CRS.from_proj4(f'+proj=aeqd +lon_0={x0} +lat_0={y0} +ellps=WGS84 +units=m +no_defs')

### quick plot to check that that looks about right
# ax=world.to_crs(aeqd).plot(figsize=[2,2], fc='none')
# ax.scatter(0,0)
# ax.set_axis_off()

## calculate the distance from this 'centroid' to the edge of thepolygon:
# first reproject it:
rprj_poly = thepolygon_4326.to_crs(aeqd)

# get distance to boundary:
d = rprj_poly.geometry[0].exterior.distance(Point(0,0))
print(f'distance from "centroid" to nearest point on boundary: {np.round(d/1000,3)} km')

# ax = rprj_poly.plot(fc='none', ec='k', label='the polygon')
# ax.scatter(0,0, label='the origin / polygon centroid approximation')
# ax.plot(*Point(0,0).buffer(d).exterior.coords.xy, label='circle of radius that just touches polygon boundary')
# ax.legend(fontsize=8)

distance from "centroid" to nearest point on boundary: 2263.504 km


In [6]:
asdf = make_circle_and_points(d,aeqd,20)
asdf['dist'] = asdf.apply(lambda row: distance_to_boundary(row['proj4'], thepolygon_4326), axis=1)

In [ ]:
for i in tqdm(range(2)):
    print(f'current search diameter: {d}')
    points = make_circle_and_points(d, aeqd, 20*i)

    points['distance_to_boundary'] = points.apply(lambda row: distance_to_boundary(row['proj4'], thepolygon_4326), axis=1)
    

    idx_max = points['distance_to_boundary'].idxmax()
    print(f"current best estimate: (lon,lat,distance (m)) {points.loc[idx_max, ['lon','lat','distance_to_boundary']]}")

    # get new aeqd projection
    aeqd = CRS.from_proj4(points.loc[points['distance_to_boundary'].idxmax(),'proj4'])

    # get difference between the current furtherst and the n-th furthest away (either 2nd, 3rd... or 5th)
    d = points.loc[idx_max,'geometry'].distance(
        points.loc[points['distance_to_boundary'].nlargest(10).index[-1],'geometry']
        )
    
    print(f"current best estimate: {points.loc[idx_max, ['lon','lat','distance_to_boundary']]}")
    
    # fig, ax = plt.subplots()
    # points.to_crs(4326).plot(column='distance_to_boundary',ax=ax)
    # asia.plot(fc='none',ax=ax)
    # ax.set_xlim(40,120)
    # ax.set_ylim(10,70)

In [ ]:
actual_poles.to_crs(points.crs)['geometry'].distance(points.loc[points['distance_to_boundary'].idxmax(),'geometry'])/1000

In [ ]:
m = points.sort_values(by='distance_to_boundary').explore(column='distance_to_boundary')
actual_poles.explore(m=m)
m

In [ ]:
m = points.explore()
actual_pole.explore(m=m)
m

In [ ]:
aeqd = CRS.from_proj4(points.loc[points['distance_to_boundary'].idxmax(),'proj4'])


In [ ]:
points.loc[points['distance_to_boundary'].nlargest(5).index].to_crs(4326).explore(column='distance_to_boundary', legend=True)

In [ ]:
ax=points.to_crs(4326).plot(column='distance_to_boundary',legend=True)
thepolygon_4326.plot(ax=ax,fc='none')

In [ ]:

# construct geoseries and reproject to geographic coordaintes
circle_4326 = gpd.GeoSeries(circle, crs=aeqd).to_crs(4326)

# get lon,lat bounds of the circle, then grid
minlon, minlat, maxlon, maxlat = circle_4326.total_bounds

In [ ]:
points = gpd.GeoSeries(points, crs=4326)
points_clipped = points.clip(largest_area)
m = points_clipped.explore()
largest_area.centroid.explore(m=m)


In [ ]:
wgs84=CRS('EPSG:4326')
aeqd = CRS.from_proj4(f'+proj=aeqd +lon_0=86.67 +lat_0=46.28 +ellps=WGS84 +units=m +no_defs')